In [1]:
import ants
import glob
from tqdm import tqdm
import pandas as pd
from utils import *
import os

<br>

<br>

### Apply transformation matrix to single image

In [147]:
# Load the image you want to transform
image_to_transform = ants.image_read('/Users/lbreitzl/Desktop/temp_lm_files/checker_board_3D.nrrd')

# Load the transformation matrices
transformation_matrix = ['/Users/lbreitzl/Desktop/temp_additional_markers/pth2_1_stitched_processed_elavl31Warp.nii.gz',
                         '/Users/lbreitzl/Desktop/temp_additional_markers/pth2_1_stitched_processed_elavl30GenericAffine.mat']

# Load a reference image (ANTs applies image dimensions from reference stack to image_to_transform stack)
reference_image = ants.image_read('/Users/lbreitzl/Desktop/temp_new_hcn1_run_brains/T_UNIL_average_brain_template0_922x922.nrrd')

In [148]:
# Apply the transformation to the image
transformed_image = ants.apply_transforms(
    fixed=reference_image,
    moving=image_to_transform,
    transformlist=transformation_matrix,
    interpolator='welchWindowedSinc'  # or 'linear', 'bspline', 'welchWindowedSinc' etc. 
)

# Save the transformed image
ants.image_write(transformed_image, '/Users/lbreitzl/Desktop/temp_lm_files/checker_board_3D_transformed.nii.gz')

<br>

<br>

### Loop over multiple files. Apply the same transformation matrix to each

In [10]:
parent = '/Users/lbreitzl/Desktop/Desktop_Kappel_HCR_brains_merged'
files = sorted(glob.glob(parent+'/landmarktrans*GCamP.nrrd'))
print(files)

In [14]:
# Load the transformation matrices (order important)
transformation_matrix = ['/Volumes/jlarsch/default/D2c/Lukas/Data/Microscopy/Analysis/Registrations/HCR_cleared_ref_to_Kappel_HCR_ref/masked_landmarktransformed_HCR_superStack_gs015_onlySyN1Warp.nii.gz',
                         '/Volumes/jlarsch/default/D2c/Lukas/Data/Microscopy/Analysis/Registrations/HCR_cleared_ref_to_Kappel_HCR_ref/masked_landmarktransformed_HCR_superStack_gs015_onlySyN0GenericAffine.mat']

# Load a reference image (it could be the same as the moving image)
reference_image = ants.image_read('/Volumes/jlarsch/default/D2c/Lukas/Data/Microscopy/Analysis/Registrations/HCR_cleared_ref_to_Kappel_HCR_ref/Gaussian3d_2_2_5_T_average_brain_hcn1_.nrrd_template0.nrrd')

for file in tqdm(files):
    # print(parent+'/'+'SyN_transformed_GCamP_'+file.split('_')[-2])
    image_to_transform = ants.image_read(file)

    # Apply the transformation to the image
    transformed_image = ants.apply_transforms(
        fixed=reference_image,
        moving=image_to_transform,
        transformlist=transformation_matrix,
        interpolator='welchWindowedSinc'  # or 'linear', 'bspline', 'welchWindowedSinc' etc. 
    )

    file_output_path = parent+'/'+'SyN_transformed_GCamP_'+file.split('_')[-2]+'.nii.gz'
    # Save the transformed image
    ants.image_write(transformed_image, file_output_path)

100%|████████████████████████████████████████| 28/28 [1:24:34<00:00, 181.23s/it]


<br>

<br>

### Loop over multiple files. Apply a corresponding transformation matrix to each

##### Enter paths to stacks and relevant information 


In [3]:
# Enter paths to stacks (brains to transform), transformation files (of brains already transformed), and reference brain
path_to_stacks = '/Volumes/jlarsch/default/D2c/Lukas/Data/Microscopy/Analysis/Reference_stacks/vglut_reference'
path_to_transformation_matrices = '/Volumes/jlarsch/default/D2c/Lukas/Data/Microscopy/Analysis/Reference_stacks/vglut_reference'
path_to_reference_brain = '/Users/lbreitzl/Desktop/temp_new_hcn1_run_brains/T_UNIL_average_brain_template0_922x922.nrrd'

# Enter the name of relevant markers. Marker names have to be included in the file name
aligned_channel = 'elavl3'         # the channel that is already aligned (usually elavl3 or GCaMP)
unaligned_channel = 'vglut'        # the channel to align (f.e. cfos, cort, pth2, etc). 
file_ending = '.nrrd'             # file ending of unaligned stacks. Could also be .tif, .nii.gz, etc. Always include dot.

##### Using above information, the following cell saves files needed for transormations as objects and determines file names of unaligned stacks (double check that those are correct)

In [5]:
# warp and transformation matrix paths saved based on respective paths and expected file names (ANTs should have created Warp.nii.gz and GenericAffine.mat file endings for these)
warp_file_paths = glob.glob(path_to_transformation_matrices+'/*1Warp.nii.gz')
transmat_file_paths = glob.glob(path_to_transformation_matrices+'/*GenericAffine.mat')
warp_file_paths.sort()
transmat_file_paths.sort()
assert len(transmat_file_paths) == len(warp_file_paths)

# Extract file names of aligned stacks and infer names of unaligned stacks. 
file_names = [x.split('/')[-1] for x in warp_file_paths]
file_names = [x.replace("1Warp.nii.gz", file_ending) for x in file_names]
unaligned_file_names = [x.replace(aligned_channel, unaligned_channel) for x in file_names]
unaligned_file_paths = [path_to_stacks+'/'+x for x in unaligned_file_names]

# Create paths + file names to save aligned brains later on
output_paths = [x.replace(file_ending, '_aligned.nii.gz') for x in unaligned_file_paths]

# unaligned_file_paths

In [6]:
warp_file_paths

['/Volumes/jlarsch/default/D2c/Lukas/Data/Microscopy/Analysis/Reference_stacks/vglut_reference/vglut_3_stitched_processed_elavl31Warp.nii.gz',
 '/Volumes/jlarsch/default/D2c/Lukas/Data/Microscopy/Analysis/Reference_stacks/vglut_reference/vglut_4_stitched_processed_elavl31Warp.nii.gz']

##### combine all paths into a single dataframe

In [7]:
alignment_files_df = pd.DataFrame([unaligned_file_paths, warp_file_paths, transmat_file_paths, output_paths]).T
alignment_files_df.columns = ['brain_to_align', 'warp_file', 'transmat_file', 'output_path']
alignment_files_df['reference_brain'] = path_to_reference_brain

alignment_files_df.head()

,brain_to_align,warp_file,transmat_file,output_path,reference_brain
0,/Volumes/jlarsch/default/D2c/Lukas/Data/Micros...,/Volumes/jlarsch/default/D2c/Lukas/Data/Micros...,/Volumes/jlarsch/default/D2c/Lukas/Data/Micros...,/Volumes/jlarsch/default/D2c/Lukas/Data/Micros...,/Users/lbreitzl/Desktop/temp_new_hcn1_run_brai...
1,/Volumes/jlarsch/default/D2c/Lukas/Data/Micros...,/Volumes/jlarsch/default/D2c/Lukas/Data/Micros...,/Volumes/jlarsch/default/D2c/Lukas/Data/Micros...,/Volumes/jlarsch/default/D2c/Lukas/Data/Micros...,/Users/lbreitzl/Desktop/temp_new_hcn1_run_brai...


In [8]:
#start_from_file = 3
#alignment_files_df = alignment_files_df.iloc[9::, :]

In [9]:
i = 0

In [15]:
alignment_files_df.brain_to_align.iloc[i]

'/Volumes/jlarsch/default/D2c/Lukas/Data/Microscopy/Analysis/Reference_stacks/vglut_reference/vglut_4_stitched_processed_vglut.nrrd'

In [16]:
alignment_files_df.warp_file.iloc[i]

'/Volumes/jlarsch/default/D2c/Lukas/Data/Microscopy/Analysis/Reference_stacks/vglut_reference/vglut_4_stitched_processed_elavl31Warp.nii.gz'

In [17]:
alignment_files_df.transmat_file.iloc[i]

'/Volumes/jlarsch/default/D2c/Lukas/Data/Microscopy/Analysis/Reference_stacks/vglut_reference/vglut_4_stitched_processed_elavl30GenericAffine.mat'

In [18]:
alignment_files_df.output_path.iloc[i]

'/Volumes/jlarsch/default/D2c/Lukas/Data/Microscopy/Analysis/Reference_stacks/vglut_reference/vglut_4_stitched_processed_vglut_aligned.nii.gz'

In [19]:
i += 1

##### apply transformations

In [140]:
for index, entry in tqdm(alignment_files_df.iterrows()):

    # Load the image to transform
    image_to_transform = ants.image_read(entry.brain_to_align)
    
    # Extract transformation matrix from dataframe. Order important!
    transformation_matrix = [entry.warp_file, entry.transmat_file]

    # Load a reference image (ANTs applies image dimensions from reference stack to image_to_transform stack)
    reference_image = ants.image_read(entry.reference_brain)

    # Apply the transformation to the image
    transformed_image = ants.apply_transforms(
        fixed=reference_image,
        moving=image_to_transform,
        transformlist=transformation_matrix,
        interpolator='welchWindowedSinc'  # or 'linear', 'bspline', 'welchWindowedSinc' etc. 
    )
    
    # Save the transformed image
    ants.image_write(transformed_image, entry.output_path)


2it [03:43, 111.77s/it]
